# RSA

Using this notebook:
- Code blocks have dark backgrounds.
- You can edit code blocks by clicking inside the block.
- You can execute a code block by clicking the "run" button (see above), or by pressing Shift+Enter while editing the block.
- The order in which code blocks are executed matters.  Normally we run from top to bottom.
- The order in which code blocks have been run follows nubmers to the left. For example, the code block with `In [1]` was the first one to be run. 

Some tips:
- To start over and execute all blocks, select Kernel -> Restart & Run All.

SAVING YOUR WORK:
- To save your work: click on the cloud icon with the downward pointing arrow.
- To restore your work: click on the cloud icon with the upward pointing arrow.
- If the system says "connection lost": save your work, open the URL again, and restore your work.

In [1]:
import sympy

First we need to choose two prime numbers, $p$ and $q$. 

To do this, typically we would start by generating a prime candidate with the number of desired bits. One way to do this is to randomly generate each bit, and then setting the first and last bits to 1. This ensures that the generated number has both:
- the number of desired bits (don't want a 0 bit in the position with the highest positional value), and 
- is an odd number (even numbers are not prime, except for 2 which is special).  

Then we need to test if the generated number is a prime number. This is not straightforward. Typically a first test would check if the prime candidate is divisible by any of the smaller primes. If it is, reject the candidate and start again.

If the prime candidate passes the first test, then a second test is performed. For very large prime candidates, it is impractical to check for divisibility of any prime number less than the candidate (and in fact for extremely large numbers we may not even know all of the prime numbers less than the candidate). This second test tells us if the proposed candidate is likely a prime number. To proceed with the assumption that it is requires that the test returns a high probability that the candidate is indeed prime.

For toy examples, we can instead use theory that tells about the structure of certain primes. Here we generate two Mersenne primes: prime numbers that have the form $2^n-1$ for some integer $n$. There is a finite list of integer exponents $n$ that will result in Mersenne primes, including $2, 3, 5, 7, 13, 17, 19, 31, \ldots$. To date there are only 51 Mersenne primes known, but it has been conjectured that there are infinitely many that exist. (Cool huh?)

In [2]:
p = (2**31) - 1
q = (2**61) - 1
print(f'p: {p}')
print(f'q: {q}')

p: 2147483647
q: 2305843009213693951


For these smaller prime numbers, we can use the function `isprime` to make sure that $p$ and $q$ are both prime:

In [3]:
print(f'p is prime: {sympy.isprime(p)}')
print(f'q is prime: {sympy.isprime(q)}')

p is prime: True
q is prime: True


Next we multiply $p$ and $q$ these to get $n$:

In [4]:
n = p * q
print(f'n: {n}')

n: 4951760154835678088235319297


Then we compute the totient of $n$, $\varphi(n)$. Recall that for a prime number $p$, $\varphi(p)=p-1$ because prime numbers are coprime with all of the $p-1$ positive integers less than themselves. Additionally, $\varphi(p\cdot q)=(p-1)\cdot (q-1)$. This means that since we know how to factor $n$ (because we constructed it), we can easily evaluate the totient of $n$.

In [5]:
phi = (q-1) * (p-1) # can also use: sympy.totient(n)
print(f'phi: {phi}')

phi: 4951760152529835076874141700


Next we choose $e$ so that $e < n$; and $gcf(e, phi(n)) = 1$.  For example, we can choose

In [6]:
e = 17
gcd = sympy.gcd(e, phi)
print(f'e: {e}; gcd: {gcd}')

e: 17; gcd: 1


Then we need to find the inverse of $e$ mod $phi(n)$. This value we are looking for is called $d$, and is part of our private key. We use the `mod_inverse` function:

In [7]:
d = sympy.mod_inverse(e, phi)
print(f'd: {d}')
print(f'check: {(d*e)%phi}')

d: 4077920125612805357425763753
check: 1


Then choose a message, like "I like math". Then map this message to ASCII characters, and take that list of numbers and make it into one large number:

In [8]:
message = "I LIKE MATH"
ordinal_message = [ ord(c) for c in message ]
large_integer_message = sum([ 100**i*o for i, o in enumerate(reversed(ordinal_message)) ])
print(f'message: {message}')
print(f'ascii ordinal version of message: {ordinal_message}')
print(f'large integer version of message: {large_integer_message}')

message: I LIKE MATH
ascii ordinal version of message: [73, 32, 76, 73, 75, 69, 32, 77, 65, 84, 72]
large integer version of message: 7332767375693277658472


Now we are ready to encrypt the message. To do this we use the power mod function which take three arguments, the base (m), the exponent (e), and the modulus (n).

In [9]:
encrypted_message = pow(large_integer_message, e, n)
print(f'encrypted message: {encrypted_message}')

encrypted message: 2079388683330327645114069204


Now we have our enciphered message. To recover our plaintext, we just use the power mod function again, but this time our base is c, the exponent is d, and the modulus is again n. 

In [10]:
decrypted_message = pow(encrypted_message, d, n)
print(f'decrypted message: {decrypted_message}')
print(f'does it match?: {decrypted_message==large_integer_message}')

decrypted message: 7332767375693277658472
does it match?: True


Finally, let's convert that large integer back into a message

In [11]:
a = decrypted_message
ords = []
while a:
    ords.append(a % 100)
    a //= 100
recovered_message = ''.join(reversed([ chr(i) for i in ords ]))
print(f'recovered message: {recovered_message}')


recovered message: I LIKE MATH
